In [2]:
import os
import pandas as pd
import numpy as np
import sys

from dotenv import load_dotenv
from openai import AzureOpenAI

from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage

import csv
import json
import time
from datetime import date, datetime

In [23]:
# Get list of frequent skills from CSV
file_path = 'C:\\Users\\Denis_Davydov2\\OneDrive - EPAM\\Prophet_AI_docs\\Datasets\\AI_skills\\Quiz\\'
file_name = 'frequent_skills_hiku45_gpt5mini_2025-11-19.csv'
frequent_skills_df = pd.read_csv(file_path+file_name)
print("File read: ", file_name)

print(f"Number of frequent skills: {len(frequent_skills_df)}")
skills_list=(frequent_skills_df['skill_name'].tolist())
print(skills_list)


File read:  frequent_skills_hiku45_gpt5mini_2025-11-19.csv
Number of frequent skills: 137
['AI Evaluation and Benchmarking', 'Model Quantization and Mixed Precision', 'LLM Integration and Deployment', 'Open-Source Model Management', 'GPU Performance Engineering', 'AI Agent Engineering', 'On-Device AI Deployment', 'AI Model Fine-Tuning', 'Generative Media Engineering', 'LLM Context Engineering', 'Multimodal AI Engineering', 'LLM Application Engineering', 'AI Performance and Cost Optimization', 'Llama Stack Engineering', 'Prompt Engineering', 'AI Safety and Governance', 'Hugging Face Transformer Engineering', 'Vision-Language Model Engineering', 'LLM Tool Integration', 'AI-Assisted Software Development', 'Conversational AI Engineering', 'AI Inference Engineering', 'Distributed Training and Inference', 'Advanced RAG Engineering', 'Speech and Vision AI', 'AI Model Serving and Deployment', 'Retrieval Systems Engineering', 'Diffusion Model Engineering', 'AI Memory Optimization', 'Efficient S

In [43]:
# Get skills list
file_path = 'C:\\Users\\Denis_Davydov2\\OneDrive - EPAM\\Prophet_AI_docs\\Datasets\\AI_skills\\Clusters\\'
file_name = 'Skills_with_names_2025-11-19.csv'
skills = pd.read_csv(file_path+file_name)
print("File read: ", file_name)
print("Total skills: ", len(skills))

skills['skill_full'] = skills['skill_name'] + ":\n" + skills['skill_definition']

# Filter skills with more than 10 occurrences
skills_filtered = skills[skills['skill_name'].isin(skills_list)].copy()
print("Filtered skills: ", len(skills_filtered))    
skills_filtered = skills_filtered[['skill_id', 'skill_name', 'skill_definition', 'skill_full']].drop_duplicates().reset_index(drop=True)
skills_filtered



File read:  Skills_with_names_2025-11-19.csv
Total skills:  144
Filtered skills:  137


,skill_id,skill_name,skill_definition,skill_full
0,42,Accelerated Tensor Programming,Develop and optimize matrix and tensor computa...,Accelerated Tensor Programming:\nDevelop and o...
1,22,Adaptive Decision Optimization,"Design, implement, and tune adaptive, stochast...","Adaptive Decision Optimization:\nDesign, imple..."
2,122,Advanced RAG Engineering,"Design, build, and optimize retrieval-augmente...","Advanced RAG Engineering:\nDesign, build, and ..."
3,28,AI 3D Content Generation,"Design, train, and deploy 3D deep learning pip...","AI 3D Content Generation:\nDesign, train, and ..."
4,10,AI Accelerator Engineering,"Architect, implement, and optimize AI accelera...","AI Accelerator Engineering:\nArchitect, implem..."
...,...,...,...,...
132,65,Time Series Predictive Modeling,"Designs, trains, and deploys time series and s...","Time Series Predictive Modeling:\nDesigns, tra..."
133,39,Transformer Attention Optimization,"Ability to design, implement, and optimize tra...",Transformer Attention Optimization:\nAbility t...
134,81,Vector Search Engineering,"Design, implement, and tune approximate neares...","Vector Search Engineering:\nDesign, implement,..."
135,56,Vision-Language Model Engineering,"Design, fine-tune, and deploy vision- and vide...","Vision-Language Model Engineering:\nDesign, fi..."


In [44]:
term_and_def_list="\n\n".join(skills_filtered['skill_full'].tolist())
print(term_and_def_list)

Accelerated Tensor Programming:
Develop and optimize matrix and tensor computations on GPUs and TPUs using SIMD/AVX2, tensor cores, GEMM, loop tiling, and PTX to maximize throughput. Leverage ONNX Runtime or TensorRT with distributed trainers like Ray Train and OneTrainer, apply dtensor for partitioning, and monitor and tune Cloud TPU workloads with TensorBoard and TPU tooling.

Adaptive Decision Optimization:
Design, implement, and tune adaptive, stochastic decision-making systems using methods like contextual bandits, MDP/MCTS, MPC, evolutionary algorithms, and Monte Carlo sampling to optimize outcomes under uncertainty. Select and configure algorithms, set decision thresholds, and deploy data-driven sampling and behavior trees to improve performance in dynamic environments.

Advanced RAG Engineering:
Design, build, and optimize retrieval-augmented generation systems across text, vision, and video, including agentic, corrective, self-RAG, and graph/RDF variants. Integrate vector and 

# Test creation

## Version 1

In [26]:
# Refined prompt aligned with JSON grouping schema
# Builds the user message used alongside system_instructions.

# Note: system_instructions already enforce JSON structure. This prompt supplies raw skills and grouping guidance.

prompt = f"""
Your task is to create a quiz for employees of a high-tech company.
Don't make the questions too complicated. Try to use simple language.
You will be presented with a list of terms and their definitions.
Create a quiz (test).
Rules:
- The number of tasks is equal to the number of terms.
- Each term has its own task
- Each task must include a question and four answer options, only one of which is correct.
- For incorrect options (distractors), use words from the correct option or from the definitions of other similar terms.
- Incorrect options should appear grammatically correct and meaningful, but in reality, they are meaningless.

Additionally, rate the task difficulty as the probability of a non-technical employee answering correctly, where 1 means everyone certainly knows the correct answer, 0 means no one knows the correct answer.

OUTPUT MUST BE ONLY JSON (enforced separately). DO NOT include explanations here.

Schema (each object):
- Skill name (get from the AI ​​skills list; Skill name only, no definition or commentary.)
- Task (question)
- Answer options (4 items, Place the correct option first.)
- Correct answer (number of the correct option, 1-4)
- Explanation (explanation of the correctness of the answer)
- Probability score for the correct answer (from 0 to 1)

Terms and definitions list:\n
{term_and_def_list}
"""

print(prompt)


Your task is to create a quiz for employees of a high-tech company.
Don't make the questions too complicated. Try to use simple language.
You will be presented with a list of terms and their definitions.
Create a quiz (test).
Rules:
- The number of tasks is equal to the number of terms.
- Each term has its own task
- Each task must include a question and four answer options, only one of which is correct.
- For incorrect options (distractors), use words from the correct option or from the definitions of other similar terms.
- Incorrect options should appear grammatically correct and meaningful, but in reality, they are meaningless.

Additionally, rate the task difficulty as the probability of a non-technical employee answering correctly, where 1 means everyone certainly knows the correct answer, 0 means no one knows the correct answer.

OUTPUT MUST BE ONLY JSON (enforced separately). DO NOT include explanations here.

Schema (each object):
- Skill name (get from the AI ​​skills list; S

In [27]:
# === Load Environment Variables ===
load_dotenv()  # Loads variables from .env file into environment

# Retrieve database connection parameters from environment variables
DIAL_API_KEY = os.getenv('DIAL_API_KEY')

#print(DIAL_API_KEY)

In [28]:
# === Utility Functions for Quiz Processing ===
import re
from datetime import datetime

def extract_json(text: str) -> str:
    """Robustly extract a JSON array substring from model output."""
    # Prefer fenced code blocks
    if '```' in text:
        parts = text.split('```')
        for part in parts:
            p = part.strip()
            if p.startswith('json'):
                p = p[4:].strip()
            if p.startswith('[') and p.endswith(']'):
                return p
    # Fallback to bracket search
    start = text.find('[')
    end = text.rfind(']')
    if start != -1 and end != -1 and end > start:
        return text[start:end+1]
    raise ValueError('JSON array delimiters not found in model output.')

def coerce_probability(val, default=0.5):
    """Convert a value to float within [0,1]; clamp if necessary."""
    try:
        f = float(val)
        if f < 0: f = 0.0
        if f > 1: f = 1.0
        return f
    except Exception:
        return float(default)

def build_explanation(skill_name: str, definition: str, provided: str | None) -> str:
    """Use provided explanation if non-empty; else synthesize from definition snippet."""
    if provided:
        expl = provided.strip()
        if expl:
            return expl
    snippet = ''
    if definition:
        sentence_end = definition.find('.')
        snippet = definition[:sentence_end+1] if 0 < sentence_end < 180 else definition[:180]
    return (f"Correct answer reflects {skill_name}: {snippet.strip()}").strip()

def ensure_four_options(opts: list[str]) -> list[str]:
    """Pad/truncate to exactly four option strings."""
    cleaned = [str(o).strip() for o in opts if str(o).strip()]
    while len(cleaned) < 4:
        cleaned.append('')
    if len(cleaned) > 4:
        cleaned = cleaned[:4]
    return cleaned

print("Utility functions loaded.")

Utility functions loaded.


In [29]:
# === Quiz Generation (Refactored for Flattened Schema) ===
import json
from datetime import datetime

# Initialize the Azure OpenAI model (already imported earlier)
model = AzureChatOpenAI(
    azure_endpoint="https://ai-proxy.lab.epam.com",
    api_key=DIAL_API_KEY,
    api_version="2025-04-01-preview",
    model='gpt-5-2025-08-07'
)

system_instructions = (
    "You are an expert in creating quizzes for AI developers and AI project managers. "
    "Task: Create an AI skills quiz for employees of a high-tech company. "
    "Return ONLY a valid JSON array (UTF-8) with objects of this exact schema:\n"
    "[\n"
    "  {\n"
    "    \"skill_name\": \"string\",\n"
    "    \"question\": \"string\",\n"
    "    \"option1\": \"string\",\n"
    "    \"option2\": \"string\",\n"
    "    \"option3\": \"string\",\n"
    "    \"option4\": \"string\",\n"
    "    \"correct_answer\": \"int\",\n"
    "    \"explanation\": \"string\",\n"
    "    \"probability\": \"float\"\n"
    "  }\n"
    "]\n\n"
    "Requirements:\n"
    "- One task per skill from the provided list.\n"
    "- Question should test understanding of the skill concept (1-2 sentences).\n"
    "- Exactly 4 options per question, only ONE marked as correct.\n"
    "- Incorrect options (distractors) should use terminology from the domain but be subtly wrong.\n"
    "- Probability: estimated chance (0.0 to 1.0) a non-technical employee answers correctly.\n"
    "- Use only double quotes; no trailing commas.\n"
    "- Exclude commentary, markdown, or code fences. Output MUST start with '[' and end with ']'."
)

messages = [
    ("system", system_instructions),
    ("human", prompt),
]

start_ts = datetime.now()
print('Invocation start:', start_ts)
output = model.invoke(messages)
raw_response = output.content.strip()
print("Raw response length:", len(raw_response))

# Extract and parse JSON
try:
    json_str = extract_json(raw_response)
    quiz_data = json.loads(json_str)
    print(f"Parsed {len(quiz_data)} items from JSON.")
except Exception as e:
    print("Primary parse failed:", e)
    cleaned = raw_response.replace('\r', ' ').replace('\n', ' ').strip('`')
    try:
        json_str = extract_json(cleaned)
        quiz_data = json.loads(json_str)
        print(f"Parsed {len(quiz_data)} items after cleaning.")
    except Exception as e2:
        print("Secondary parse failed:", e2)
        quiz_data = []

skill_def_map = {row.skill_name: row.skill_definition for row in skills_filtered[['skill_name','skill_definition']].itertuples()}
all_skill_names = set(skills_filtered['skill_name'].tolist())
rows = []

for idx, item in enumerate(quiz_data):
    if not isinstance(item, dict):
        continue
    skill_name = str(item.get('skill_name','')).strip()
    question = str(item.get('question','')).strip()
    # Collect option fields (flattened schema)
    options_flat = [item.get('option1',''), item.get('option2',''), item.get('option3',''), item.get('option4','')]
    options_flat = ensure_four_options(options_flat)

    # Determine correct answer index (expecting 1-4); force 1 if invalid
    try:
        correct_answer_raw = int(item.get('correct_answer', 1))
        if correct_answer_raw not in (1,2,3,4):
            correct_answer_raw = 1
    except Exception:
        correct_answer_raw = 1

    # If correct answer not first, rotate so option1 becomes correct and preserve text
    if correct_answer_raw != 1:
        corr_idx = correct_answer_raw - 1
        options_flat[0], options_flat[corr_idx] = options_flat[corr_idx], options_flat[0]
        correct_answer_raw = 1

    probability_val = coerce_probability(item.get('probability', 0.5))
    provided_expl = item.get('explanation','')
    explanation = build_explanation(skill_name, skill_def_map.get(skill_name,''), provided_expl)

    if not skill_name or not question:
        print(f"Warning: skipping incomplete row {idx}")
        continue

    rows.append({
        'skill_name': skill_name,
        'question': question,
        'option_1': options_flat[0],
        'option_2': options_flat[1],
        'option_3': options_flat[2],
        'option_4': options_flat[3],
        'correct_answer': 1,
        'probability': probability_val,
        'in_skills_list': skill_name in all_skill_names,
        'Explanation': explanation
    })

# Coverage enforcement: add placeholder tasks for any missing skills
present_skills = {r['skill_name'] for r in rows}
missing_skills = sorted(list(all_skill_names - present_skills))
if missing_skills:
    print(f"Adding {len(missing_skills)} placeholder questions for missing skills.")
    for ms in missing_skills:
        definition = skill_def_map.get(ms,'')
        placeholder_q = f"Which statement best reflects the concept of {ms}?"[:180]
        base_expl = build_explanation(ms, definition, None)
        rows.append({
            'skill_name': ms,
            'question': placeholder_q,
            'option_1': f"Correct concise definition of {ms}",
            'option_2': f"Overly broad misinterpretation of {ms}",
            'option_3': f"Irrelevant technology unrelated to {ms}",
            'option_4': f"Historically outdated form of {ms}",
            'correct_answer': 1,
            'probability': 0.4,
            'in_skills_list': True,
            'Explanation': base_expl
        })

# Build final DataFrame
df_quiz = pd.DataFrame(rows)
end_ts = datetime.now()
print('\n' + '='*80)
print('QUIZ GENERATION SUMMARY (Refactored)')
print('='*80)
print('Total rows:', len(df_quiz))
print('Unique skills represented:', df_quiz['skill_name'].nunique())
print('Missing skill placeholders added:', len(missing_skills))
if not df_quiz.empty:
    print(f"Avg probability: {df_quiz['probability'].mean():.3f}")
    print('Option1 empty count:', (df_quiz['option_1']=='' ).sum())
print('Processing time:', end_ts - start_ts)
print('='*80)

df_quiz

Invocation start: 2025-11-20 02:09:40.111296
Raw response length: 28161
Parsed 48 items from JSON.
Adding 89 placeholder questions for missing skills.

QUIZ GENERATION SUMMARY (Refactored)
Total rows: 137
Unique skills represented: 137
Missing skill placeholders added: 89
Avg probability: 0.446
Option1 empty count: 0
Processing time: 0:00:56.715224
Raw response length: 28161
Parsed 48 items from JSON.
Adding 89 placeholder questions for missing skills.

QUIZ GENERATION SUMMARY (Refactored)
Total rows: 137
Unique skills represented: 137
Missing skill placeholders added: 89
Avg probability: 0.446
Option1 empty count: 0
Processing time: 0:00:56.715224


,skill_name,question,option_1,option_2,option_3,option_4,correct_answer,probability,in_skills_list,Explanation
0,Accelerated Tensor Programming,What does accelerated tensor programming aim t...,Run matrix and tensor math very fast on GPUs/T...,Slow down GPU math by turning off SIMD and usi...,Replace GEMM with random loops so tensors beco...,Use Ray Train to disable AVX2 so Cloud TPU job...,1,0.38,True,Accelerated tensor programming is about maximi...
1,Adaptive Decision Optimization,What is the goal of adaptive decision optimiza...,Choose actions under uncertainty by tuning ban...,Freeze decisions to fixed thresholds so stocha...,Use evolutionary algorithms only for logging w...,Pick random actions and store them in a behavi...,1,0.42,True,Adaptive decision optimization uses data-drive...
2,Advanced RAG Engineering,What does an advanced RAG system do?,Combines retrieval from vector/graph stores wi...,Removes retrieval entirely so the generator gu...,Stores documents only in RAM without embedding...,Uses RAFT only to rename indexes without chang...,1,0.40,True,RAG augments generation with relevant retrieve...
3,AI 3D Content Generation,What is AI 3D content generation used for?,Reconstruct and create 3D assets and scenes fr...,Convert 3D scenes into spreadsheets so depth e...,Use generative rendering to flatten images int...,Train avatars by removing animation from the p...,1,0.46,True,The skill focuses on building 3D pipelines tha...
4,AI Accelerator Engineering,What is the core focus of AI accelerator engin...,Design and optimize custom chips and firmware ...,Disable memory hierarchies so accelerators use...,Use chiplets only for labeling boards without ...,Write firmware that turns NPUs into passive se...,1,0.35,True,It is about creating and tuning AI hardware ar...
...,...,...,...,...,...,...,...,...,...,...
132,Time Series Predictive Modeling,Which statement best reflects the concept of T...,Correct concise definition of Time Series Pred...,Overly broad misinterpretation of Time Series ...,Irrelevant technology unrelated to Time Series...,Historically outdated form of Time Series Pred...,1,0.40,True,Correct answer reflects Time Series Predictive...
133,Transformer Attention Optimization,Which statement best reflects the concept of T...,Correct concise definition of Transformer Atte...,Overly broad misinterpretation of Transformer ...,Irrelevant technology unrelated to Transformer...,Historically outdated form of Transformer Atte...,1,0.40,True,Correct answer reflects Transformer Attention ...
134,Vector Search Engineering,Which statement best reflects the concept of V...,Correct concise definition of Vector Search En...,Overly broad misinterpretation of Vector Searc...,Irrelevant technology unrelated to Vector Sear...,Historically outdated form of Vector Search En...,1,0.40,True,Correct answer reflects Vector Search Engineer...
135,Vision-Language Model Engineering,Which statement best reflects the concept of V...,Correct concise definition of Vision-Language ...,Overly broad misinterpretation of Vision-Langu...,Irrelevant technology unrelated to Vision-Lang...,Historically outdated form of Vision-Language ...,1,0.40,True,Correct answer reflects Vision-Language Model ...


In [31]:
# === Quiz Export & Validation ===
from datetime import datetime
import json

if 'df_quiz' not in globals() or df_quiz.empty:
    print('df_quiz is empty or undefined; nothing to export.')
else:
    timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    export_dir = r'C:\Users\Denis_Davydov2\OneDrive - EPAM\Prophet_AI_docs\Datasets\AI_skills\Quiz'
    csv_path = export_dir + f'\\quiz_generated_{timestamp}.csv'
    json_path = export_dir + f'quiz_generated_{timestamp}.json'

    # Ensure columns order
    ordered_cols = [
        'skill_name','question','option_1','option_2','option_3','option_4',
        'correct_answer','probability','Explanation','in_skills_list'
    ]
    existing_cols = [c for c in ordered_cols if c in df_quiz.columns]
    df_quiz[existing_cols].to_csv(csv_path, index=False)
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(df_quiz[existing_cols].to_dict(orient='records'), f, ensure_ascii=False, indent=2)

    # Basic validation stats
    missing_skill_count = (~df_quiz['in_skills_list']).sum()
    empty_options = {
        'option_1': (df_quiz['option_1']=='').sum(),
        'option_2': (df_quiz['option_2']=='').sum(),
        'option_3': (df_quiz['option_3']=='').sum(),
        'option_4': (df_quiz['option_4']=='').sum(),
    }
    print('='*80)
    print('QUIZ EXPORT COMPLETE')
    print('='*80)
    print('CSV:', csv_path)
    print('JSON:', json_path)
    print('Rows exported:', len(df_quiz))
    print('Unique skills:', df_quiz['skill_name'].nunique())
    print('Avg probability:', round(df_quiz['probability'].mean(),3))
    print('Missing skill flag count:', missing_skill_count)
    print('Empty option counts:', empty_options)
    print('='*80)

QUIZ EXPORT COMPLETE
CSV: C:\Users\Denis_Davydov2\OneDrive - EPAM\Prophet_AI_docs\Datasets\AI_skills\Quiz\quiz_generated_2025-11-20_02-18-59.csv
JSON: C:\Users\Denis_Davydov2\OneDrive - EPAM\Prophet_AI_docs\Datasets\AI_skills\Quizquiz_generated_2025-11-20_02-18-59.json
Rows exported: 137
Unique skills: 137
Avg probability: 0.446
Missing skill flag count: 0
Empty option counts: {'option_1': np.int64(0), 'option_2': np.int64(0), 'option_3': np.int64(0), 'option_4': np.int64(0)}


## Version 2

In [55]:
# === Stricter Coverage Prompt ===
# CRITICAL: skills_ordered must be a LIST of skill names (strings), not a giant string!
skills_ordered = skills_filtered['skill_name'].tolist()
required_count = len(skills_ordered)

print(f'DEBUG: skills_ordered is a {type(skills_ordered)} with {len(skills_ordered)} items')
print(f'DEBUG: First 3 items: {skills_ordered[:3]}')

# Build formatted list with name + definition for context
skills_with_defs = []
for _, row in skills_filtered.iterrows():
    skills_with_defs.append(f"{row['skill_name']}:\n{row['skill_definition']}")

skills_context = "\n\n".join(skills_with_defs)

coverage_prompt = f"""
You must generate a JSON array with EXACTLY {required_count} objects.
Each object corresponds to one skill in the list below (same order, no omissions, no extras).

For EACH skill:
- Provide a concise multiple-choice question (aim at high-tech company employees) testing specific recognition.
- Provide four options: exactly one correct.
- The answer options should be approximately equal in length.
- For a task, add the context of artificial intelligence development and implementation.
- Keep language simple and avoid rare technical terms and abbreviations.
- Do NOT merge skills. One skill per object.
- To identify a skill_name, use the the exact string verbatim (the text before the colon).
- To create a task, use the skill name (text before the colon) and the skill definition (text after the colon)
- Probability: honest estimate (0.0-1.0) of a technical employee guessing correctly.
- Explanation: 1-2 sentences clarifying why the correct option is right.

Return ONLY JSON array, no comments, no markdown. Start with '[' end with ']'.
Schema keys: skill_name, question, option1, option2, option3, option4, correct_answer (1-4), explanation, probability.

Skills list with definitions:\n\n
{skills_context}
"""
print('Stricter coverage prompt ready.')
print(f'Required skill count: {required_count}')
print(f'Skills context length: {len(skills_context)} characters')

DEBUG: skills_ordered is a <class 'list'> with 137 items
DEBUG: First 3 items: ['Accelerated Tensor Programming', 'Adaptive Decision Optimization', 'Advanced RAG Engineering']
Stricter coverage prompt ready.
Required skill count: 137
Skills context length: 53820 characters


In [56]:
print(coverage_prompt)


You must generate a JSON array with EXACTLY 137 objects.
Each object corresponds to one skill in the list below (same order, no omissions, no extras).

For EACH skill:
- Provide a concise multiple-choice question (aim at high-tech company employees) testing specific recognition.
- Provide four options: exactly one correct.
- The answer options should be approximately equal in length.
- For a task, add the context of artificial intelligence development and implementation.
- Keep language simple and avoid rare technical terms and abbreviations.
- Do NOT merge skills. One skill per object.
- To identify a skill_name, use the the exact string verbatim (the text before the colon).
- To create a task, use the skill name (text before the colon) and the skill definition (text after the colon)
- Probability: honest estimate (0.0-1.0) of a technical employee guessing correctly.
- Explanation: 1-2 sentences clarifying why the correct option is right.

Return ONLY JSON array, no comments, no mark

In [57]:
system_instructions = (
    "You are an expert in creating quizzes for AI developers and AI project managers. "
    "Task: Create an AI skills quiz for employees of a high-tech company. "
    "Return ONLY a valid JSON array (UTF-8) with objects of this exact schema:\n"
    "[\n"
    "  {\n"
    "    \"skill_name\": \"string\",\n"
    "    \"question\": \"string\",\n"
    "    \"option1\": \"string\",\n"
    "    \"option2\": \"string\",\n"
    "    \"option3\": \"string\",\n"
    "    \"option4\": \"string\",\n"
    "    \"correct_answer\": \"int\",\n"
    "    \"explanation\": \"string\",\n"
    "    \"probability\": \"float\"\n"
    "  }\n"
    "]\n\n"
    "Requirements:\n"
    "- One task per skill from the provided list.\n"
    "- Question should test understanding of the skill concept (1-2 sentences).\n"
    "- Exactly 4 options per question, only ONE marked as correct.\n"
    "- Incorrect options (distractors) should use terminology from the domain but be subtly wrong.\n"
    "- Probability: estimated chance (0.0 to 1.0) a technical employee answers correctly.\n"
    "- Use only double quotes; no trailing commas.\n"
    "- Exclude commentary, markdown, or code fences. Output MUST start with '[' and end with ']'."
)

In [58]:
# === Full Coverage Generation (Phase 1 + Fallback) ===
from datetime import datetime
import json

# Initialize the Azure OpenAI model (already imported earlier)
model = AzureChatOpenAI(
    azure_endpoint="https://ai-proxy.lab.epam.com",
    api_key=DIAL_API_KEY,
    api_version="2025-04-01-preview",
    model='gpt-5-2025-08-07'
)

# Phase 1: initial full generation
messages_cov = [
    ("system", system_instructions),
    ("human", coverage_prompt),
]
phase1_start = datetime.now()
print('Phase 1: Invoking model...')
resp1 = model.invoke(messages_cov)
raw1 = resp1.content.strip()
print('Phase 1 raw length:', len(raw1))

try:
    json_phase1 = extract_json(raw1)
    data_phase1 = json.loads(json_phase1)
    print('Phase 1 parsed items:', len(data_phase1))
except Exception as e:
    print('Phase 1 parse failed:', e)
    data_phase1 = []

# Normalize phase1
rows_phase1 = []
seen_skills = set()
for obj in data_phase1:
    if not isinstance(obj, dict):
        continue
    skill_name = str(obj.get('skill_name','')).strip()
    if not skill_name:
        continue
    if skill_name in seen_skills:
        continue  # skip duplicates
    question = str(obj.get('question','')).strip()
    options_flat = [obj.get('option1',''), obj.get('option2',''), obj.get('option3',''), obj.get('option4','')]
    options_flat = ensure_four_options(options_flat)
    try:
        ca = int(obj.get('correct_answer',1))
        if ca not in (1,2,3,4):
            ca = 1
    except Exception:
        ca = 1
    if ca != 1:
        corr_idx = ca - 1
        options_flat[0], options_flat[corr_idx] = options_flat[corr_idx], options_flat[0]
        ca = 1
    prob = coerce_probability(obj.get('probability',0.5))
    expl = build_explanation(skill_name, skill_def_map.get(skill_name,''), obj.get('explanation',''))
    rows_phase1.append({
        'skill_name': skill_name,
        'question': question,
        'option_1': options_flat[0],
        'option_2': options_flat[1],
        'option_3': options_flat[2],
        'option_4': options_flat[3],
        'correct_answer': 1,
        'probability': prob,
        'Explanation': expl
    })
    seen_skills.add(skill_name)

missing_after_phase1 = [s for s in skills_ordered if s not in seen_skills]
print('Missing after phase 1:', len(missing_after_phase1))

# Phase 2: fallback for missing skills only
rows_phase2 = []
if missing_after_phase1:
    print(f'Phase 2: Generating {len(missing_after_phase1)} missing skills...')
    # Build context for missing skills
    missing_with_defs = []
    for skill_name in missing_after_phase1:
        definition = skill_def_map.get(skill_name, '')
        missing_with_defs.append(f"{skill_name}:\n{definition}")
    
    fallback_context = "\n\n".join(missing_with_defs)
    fallback_prompt = f"""
Generate JSON array with one object per skill below (order preserved). SAME schema as before.
Use exact skill names (text before colon).

Skills with definitions:
{fallback_context}

Return ONLY JSON array.
"""
    messages_fallback = [
        ("system", system_instructions),
        ("human", fallback_prompt),
    ]
    resp2 = model.invoke(messages_fallback)
    raw2 = resp2.content.strip()
    print('Phase 2 raw length:', len(raw2))
    try:
        json_phase2 = extract_json(raw2)
        data_phase2 = json.loads(json_phase2)
        print('Phase 2 parsed items:', len(data_phase2))
    except Exception as e:
        print('Phase 2 parse failed:', e)
        data_phase2 = []
    for obj in data_phase2:
        if not isinstance(obj, dict):
            continue
        skill_name = str(obj.get('skill_name','')).strip()
        if not skill_name or skill_name in seen_skills:
            continue
        question = str(obj.get('question','')).strip()
        options_flat = [obj.get('option1',''), obj.get('option2',''), obj.get('option3',''), obj.get('option4','')]
        options_flat = ensure_four_options(options_flat)
        try:
            ca = int(obj.get('correct_answer',1))
            if ca not in (1,2,3,4):
                ca = 1
        except Exception:
            ca = 1
        if ca != 1:
            corr_idx = ca - 1
            options_flat[0], options_flat[corr_idx] = options_flat[corr_idx], options_flat[0]
            ca = 1
        prob = coerce_probability(obj.get('probability',0.5))
        expl = build_explanation(skill_name, skill_def_map.get(skill_name,''), obj.get('explanation',''))
        rows_phase2.append({
            'skill_name': skill_name,
            'question': question,
            'option_1': options_flat[0],
            'option_2': options_flat[1],
            'option_3': options_flat[2],
            'option_4': options_flat[3],
            'correct_answer': 1,
            'probability': prob,
            'Explanation': expl
        })
        seen_skills.add(skill_name)

# Combine preserving original order
def order_rows(rows_all):
    by_skill = {r['skill_name']: r for r in rows_all}
    ordered = [by_skill[s] for s in skills_ordered if s in by_skill]
    return ordered

combined_rows = order_rows(rows_phase1 + rows_phase2)
df_quiz_full = pd.DataFrame(combined_rows)

print('='*80)
print('COVERAGE RESULT SUMMARY')
print('='*80)
print('Required skills:', required_count)
print('Generated rows:', len(df_quiz_full))
print('Unique skills:', df_quiz_full['skill_name'].nunique())
missing_final = [s for s in skills_ordered if s not in set(df_quiz_full['skill_name'])]
print('Missing after fallback:', len(missing_final))
if missing_final:
    print('Remaining missing skills:', missing_final[:10])
    if len(missing_final) > 10:
        print(f'... and {len(missing_final) - 10} more')
print('Avg probability:', round(df_quiz_full['probability'].mean(),3) if not df_quiz_full.empty else 'N/A')
phase1_end = datetime.now()
print('Total processing time:', phase1_end - phase1_start)
print('='*80)

df_quiz_full.head()

Phase 1: Invoking model...
Phase 1 raw length: 78624
Phase 1 parsed items: 137
Missing after phase 1: 0
COVERAGE RESULT SUMMARY
Required skills: 137
Generated rows: 137
Unique skills: 137
Missing after fallback: 0
Avg probability: 0.736
Total processing time: 0:02:39.919225


,skill_name,question,option_1,option_2,option_3,option_4,correct_answer,probability,Explanation
0,Accelerated Tensor Programming,"In an AI inference service on GPUs, what does ...","Using tensor cores, fused GEMM, tiling, and PT...",Refactoring Python loops and pandas joins to s...,Encrypting checkpoints and rotating IAM keys t...,Extending context windows via retrieval to imp...,1,0.70,It focuses on low-level GPU math and kernel op...
1,Adaptive Decision Optimization,"When deploying an AI system under uncertainty,...","Using contextual bandits, MDPs, and MPC to tun...",Selecting static thresholds from historical A/...,Encoding business rules in fixed if-else trees...,Scaling batch inference by adding GPUs without...,1,0.62,"It uses stochastic methods like bandits, MDPs,..."
2,Advanced RAG Engineering,"In production LLM apps, what defines Advanced ...","Using agentic, corrective, and graph-aware ret...",Only increasing embedding dimensions to improv...,Relying on static FAQs without vector or graph...,Moving prompts to serverless functions to redu...,1,0.66,Advanced RAG integrates vector and graph store...
3,AI 3D Content Generation,For AI-driven 3D asset creation from 2D inputs...,Training 3D neural models with depth estimatio...,Compressing textures with ZIP to reduce asset ...,Hosting static GLTF files on a CDN with cache ...,Tracking user clicks to optimize UX funnels in...,1,0.63,"It focuses on 3D deep learning, depth inferenc..."
4,AI Accelerator Engineering,"In building custom AI chips, what is a primary...",Designing compute and memory hierarchies with ...,Creating React dashboards for model observabil...,Writing high-level prompts to guide chat agent...,Configuring IAM groups to manage developer per...,1,0.59,"It targets chip architecture, memory, firmware..."


In [59]:
# === Final Coverage Validation & Export ===
from datetime import datetime
import json

if 'df_quiz_full' not in globals() or df_quiz_full.empty:
    print('df_quiz_full not available; run coverage generation cell first.')
else:
    missing_final = [s for s in skills_ordered if s not in set(df_quiz_full['skill_name'])]
    if missing_final:
        print('WARNING: Some skills still missing:', len(missing_final))
    else:
        print('All skills covered. No placeholders used.')


        


All skills covered. No placeholders used.


In [60]:
df_quiz_full

,skill_name,question,option_1,option_2,option_3,option_4,correct_answer,probability,Explanation
0,Accelerated Tensor Programming,"In an AI inference service on GPUs, what does ...","Using tensor cores, fused GEMM, tiling, and PT...",Refactoring Python loops and pandas joins to s...,Encrypting checkpoints and rotating IAM keys t...,Extending context windows via retrieval to imp...,1,0.70,It focuses on low-level GPU math and kernel op...
1,Adaptive Decision Optimization,"When deploying an AI system under uncertainty,...","Using contextual bandits, MDPs, and MPC to tun...",Selecting static thresholds from historical A/...,Encoding business rules in fixed if-else trees...,Scaling batch inference by adding GPUs without...,1,0.62,"It uses stochastic methods like bandits, MDPs,..."
2,Advanced RAG Engineering,"In production LLM apps, what defines Advanced ...","Using agentic, corrective, and graph-aware ret...",Only increasing embedding dimensions to improv...,Relying on static FAQs without vector or graph...,Moving prompts to serverless functions to redu...,1,0.66,Advanced RAG integrates vector and graph store...
3,AI 3D Content Generation,For AI-driven 3D asset creation from 2D inputs...,Training 3D neural models with depth estimatio...,Compressing textures with ZIP to reduce asset ...,Hosting static GLTF files on a CDN with cache ...,Tracking user clicks to optimize UX funnels in...,1,0.63,"It focuses on 3D deep learning, depth inferenc..."
4,AI Accelerator Engineering,"In building custom AI chips, what is a primary...",Designing compute and memory hierarchies with ...,Creating React dashboards for model observabil...,Writing high-level prompts to guide chat agent...,Configuring IAM groups to manage developer per...,1,0.59,"It targets chip architecture, memory, firmware..."
...,...,...,...,...,...,...,...,...,...
132,Time Series Predictive Modeling,"For demand forecasts, what is a core task?",Train temporal models and convert forecasts in...,Treat timestamps as unordered IDs to simplify ...,Use static averages for all future periods,Ignore uncertainty when planning inventory,1,0.77,Time series modeling forecasts and drives acti...
133,Transformer Attention Optimization,"To speed attention on long sequences, what helps?","FlashAttention, efficient masking, and kernel ...",Disabling attention caches to avoid memory fra...,Always using full quadratic attention regardle...,Converting tokens to images to speed kernels,1,0.72,Efficient mechanisms and tuned kernels improve...
134,Vector Search Engineering,"For scalable ANN search, what is essential?",HNSW or IVF indexes with filters and low-laten...,Linear scans over raw vectors for all queries,Indexing only metadata without vector fields,Using KNN on CPU with no batching or shards,1,0.79,ANN indexes and tuned deployments deliver rele...
135,Vision-Language Model Engineering,For document understanding with images and tex...,Fine-tune VLMs and optimize inference for inte...,Use only text models and ignore images entirely,Attach captions post-hoc without model alignment,Resize images to thumbnails for all tasks,1,0.73,VLM engineering aligns visual and language inp...


In [61]:
# Add skill_definition column from skills_filtered
df_quiz_full = df_quiz_full.merge(
    skills_filtered[['skill_name', 'skill_definition']], 
    on='skill_name', 
    how='left'
)

print(f"Added skill_definition column")
print(f"Rows with definition: {df_quiz_full['skill_definition'].notna().sum()}/{len(df_quiz_full)}")
df_quiz_full

Added skill_definition column
Rows with definition: 137/137


,skill_name,question,option_1,option_2,option_3,option_4,correct_answer,probability,Explanation,skill_definition
0,Accelerated Tensor Programming,"In an AI inference service on GPUs, what does ...","Using tensor cores, fused GEMM, tiling, and PT...",Refactoring Python loops and pandas joins to s...,Encrypting checkpoints and rotating IAM keys t...,Extending context windows via retrieval to imp...,1,0.70,It focuses on low-level GPU math and kernel op...,Develop and optimize matrix and tensor computa...
1,Adaptive Decision Optimization,"When deploying an AI system under uncertainty,...","Using contextual bandits, MDPs, and MPC to tun...",Selecting static thresholds from historical A/...,Encoding business rules in fixed if-else trees...,Scaling batch inference by adding GPUs without...,1,0.62,"It uses stochastic methods like bandits, MDPs,...","Design, implement, and tune adaptive, stochast..."
2,Advanced RAG Engineering,"In production LLM apps, what defines Advanced ...","Using agentic, corrective, and graph-aware ret...",Only increasing embedding dimensions to improv...,Relying on static FAQs without vector or graph...,Moving prompts to serverless functions to redu...,1,0.66,Advanced RAG integrates vector and graph store...,"Design, build, and optimize retrieval-augmente..."
3,AI 3D Content Generation,For AI-driven 3D asset creation from 2D inputs...,Training 3D neural models with depth estimatio...,Compressing textures with ZIP to reduce asset ...,Hosting static GLTF files on a CDN with cache ...,Tracking user clicks to optimize UX funnels in...,1,0.63,"It focuses on 3D deep learning, depth inferenc...","Design, train, and deploy 3D deep learning pip..."
4,AI Accelerator Engineering,"In building custom AI chips, what is a primary...",Designing compute and memory hierarchies with ...,Creating React dashboards for model observabil...,Writing high-level prompts to guide chat agent...,Configuring IAM groups to manage developer per...,1,0.59,"It targets chip architecture, memory, firmware...","Architect, implement, and optimize AI accelera..."
...,...,...,...,...,...,...,...,...,...,...
132,Time Series Predictive Modeling,"For demand forecasts, what is a core task?",Train temporal models and convert forecasts in...,Treat timestamps as unordered IDs to simplify ...,Use static averages for all future periods,Ignore uncertainty when planning inventory,1,0.77,Time series modeling forecasts and drives acti...,"Designs, trains, and deploys time series and s..."
133,Transformer Attention Optimization,"To speed attention on long sequences, what helps?","FlashAttention, efficient masking, and kernel ...",Disabling attention caches to avoid memory fra...,Always using full quadratic attention regardle...,Converting tokens to images to speed kernels,1,0.72,Efficient mechanisms and tuned kernels improve...,"Ability to design, implement, and optimize tra..."
134,Vector Search Engineering,"For scalable ANN search, what is essential?",HNSW or IVF indexes with filters and low-laten...,Linear scans over raw vectors for all queries,Indexing only metadata without vector fields,Using KNN on CPU with no batching or shards,1,0.79,ANN indexes and tuned deployments deliver rele...,"Design, implement, and tune approximate neares..."
135,Vision-Language Model Engineering,For document understanding with images and tex...,Fine-tune VLMs and optimize inference for inte...,Use only text models and ignore images entirely,Attach captions post-hoc without model alignment,Resize images to thumbnails for all tasks,1,0.73,VLM engineering aligns visual and language inp...,"Design, fine-tune, and deploy vision- and vide..."


In [67]:
# Add task_id column as row index (1-based)
df_quiz_full.insert(0, 'task_id', range(1, len(df_quiz_full) + 1))

print(f"Added task_id column (1-{len(df_quiz_full)})")
df_quiz_full.head()

Added task_id column (1-137)


,task_id,skill_name,question,option_1,option_2,option_3,option_4,correct_answer,probability,Explanation,skill_definition
0,1,Accelerated Tensor Programming,"In an AI inference service on GPUs, what does ...","Using tensor cores, fused GEMM, tiling, and PT...",Refactoring Python loops and pandas joins to s...,Encrypting checkpoints and rotating IAM keys t...,Extending context windows via retrieval to imp...,1,0.70,It focuses on low-level GPU math and kernel op...,Develop and optimize matrix and tensor computa...
1,2,Adaptive Decision Optimization,"When deploying an AI system under uncertainty,...","Using contextual bandits, MDPs, and MPC to tun...",Selecting static thresholds from historical A/...,Encoding business rules in fixed if-else trees...,Scaling batch inference by adding GPUs without...,1,0.62,"It uses stochastic methods like bandits, MDPs,...","Design, implement, and tune adaptive, stochast..."
2,3,Advanced RAG Engineering,"In production LLM apps, what defines Advanced ...","Using agentic, corrective, and graph-aware ret...",Only increasing embedding dimensions to improv...,Relying on static FAQs without vector or graph...,Moving prompts to serverless functions to redu...,1,0.66,Advanced RAG integrates vector and graph store...,"Design, build, and optimize retrieval-augmente..."
3,4,AI 3D Content Generation,For AI-driven 3D asset creation from 2D inputs...,Training 3D neural models with depth estimatio...,Compressing textures with ZIP to reduce asset ...,Hosting static GLTF files on a CDN with cache ...,Tracking user clicks to optimize UX funnels in...,1,0.63,"It focuses on 3D deep learning, depth inferenc...","Design, train, and deploy 3D deep learning pip..."
4,5,AI Accelerator Engineering,"In building custom AI chips, what is a primary...",Designing compute and memory hierarchies with ...,Creating React dashboards for model observabil...,Writing high-level prompts to guide chat agent...,Configuring IAM groups to manage developer per...,1,0.59,"It targets chip architecture, memory, firmware...","Architect, implement, and optimize AI accelera..."


In [69]:

ts = datetime.now().strftime('%Y_%m_%d')
export_dir = r'C:\Users\Denis_Davydov2\OneDrive - EPAM\Prophet_AI_docs\Datasets\AI_skills\Quiz'
csv_path = export_dir + f'\\quiz_v2_{ts}.csv'
json_path = export_dir + f'\\quiz_v2_{ts}.json'

ordered_cols = ['task_id', 'skill_name','question','option_1','option_2','option_3','option_4','correct_answer','probability','Explanation', 'skill_definition']
df_quiz_full[ordered_cols].to_csv(csv_path, index=False)
with open(json_path,'w',encoding='utf-8') as f:
	json.dump(df_quiz_full[ordered_cols].to_dict(orient='records'), f, ensure_ascii=False, indent=2)

print('Exported to:')
print(csv_path)
print(json_path)
print('Average probability:', round(df_quiz_full['probability'].mean(),3))
print('Empty option counts:', {c:(df_quiz_full[c]=='').sum() for c in ['option_1','option_2','option_3','option_4']})

Exported to:
C:\Users\Denis_Davydov2\OneDrive - EPAM\Prophet_AI_docs\Datasets\AI_skills\Quiz\quiz_v2_2025_11_21.csv
C:\Users\Denis_Davydov2\OneDrive - EPAM\Prophet_AI_docs\Datasets\AI_skills\Quiz\quiz_v2_2025_11_21.json
Average probability: 0.736
Empty option counts: {'option_1': np.int64(0), 'option_2': np.int64(0), 'option_3': np.int64(0), 'option_4': np.int64(0)}
